## WTP for insurance

Here I follow the notes on [Selection Markets](http://www.andreveiga.com) posted by [Andre Veiga](http://www.andreveiga.com).

**Notation**:
- $w_0$: initial wealth
- $w$: final wealth
- $\theta$: vector of characteristics describing individual type
- $u(w; \theta)$: vNM utility index for type $\theta$
- $l\in \mathbf{R}$: random wealth shock
- $g(l; \theta)$: pdf of wealth shock
- $G(l, x)$: insurer payment if loss is $l$
    - $G(l, x) \equiv l$ for full insurance
    - $G(l, x) \equiv 0$ for no insurance
- $x$: parametrizes generosity of insurance

We assume for now that there is no moral hazard: $g(l, \theta)$ independent of $x$.

Willingness to pay for insurance contract paying $G$ is the price $p=v(x, \theta)$ that makes the consumer indifferent between buying and not buying the contract:

$$ \mathbf{E}_l \Big[ u(w_0 - l + G(l, x)-v(x, \theta); \theta) | \theta \Big] = \mathbf{E}_l \Big[ u(w_0 - l ; \theta) | \theta \Big]$$

**Parametrization**

- CARA preferences: $u(c)=-e^{-ac}$
- Gaussian wealth shocks: $l \sim \mathcal{N}(\mu, \sigma^2)$
- Co-insurance: insurers absorb a share $x\in(0, 1)$ of the shock: $G(l, x) = xl$

To calculate $v(x, \theta)$, note that

$\mathbf{E}_l \Big[ exp(a \times l)\Big] = \exp\Big(\frac{a^2\sigma^2}{2}+a\mu\Big)$

(This analytical solution to the integral is not trivial - see Gaussian Integrals in Wikipedia)



WTP $v(x, \theta)$ is implicitly defined by 


$$ e^{-a(w_0-v)}\mathbf{E}_l \Big[ e^{a(1-x)l}\Big] = e^{-aw_0}\mathbf{E}_l \Big[ e^{al}\Big]$$

$$ \Leftrightarrow$$

$$ e^{av}\mathbf{E}_l \Big[ e^{a(1-x)l}\Big] = \mathbf{E}_l \Big[ e^{al}\Big]$$

$$ \Leftrightarrow$$

$$ \exp\Big(av + \frac{a^2(1-x)^2\sigma^2}{2}+a(1-x)\mu\Big) = \exp\Big(\frac{a^2\sigma^2}{2}+a\mu\Big) $$

$$ \Leftrightarrow$$

$$ v = x\mu + \frac{1}{2}a\sigma^2(1-(1-x)^2)$$




**Import libraries**

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import norm
from scipy.integrate import quad
from scipy.optimize import fsolve
from scipy.special import roots_hermite

**Consumer class**

In [17]:
class Consumer():
    """Consumer is described by:
        - λ: risk aversion parameter of CARA utility, default is 2
        - μ: mean of loss distribution, default is 3
        - σ: std of loss distribution, default is .3
        - w0: initial wealth, default is 10
    For setting w_=0 it is important to be mindfull of the scale to avoid overflow"""

    def __init__(self, 
                 λ = 2,
                 μ = 3,
                 σ = .3,
                 w0 = 10):
        # Unpack parameters
        self.λ, self.μ, self.σ , self.w0 = λ , μ , σ , w0

    def u(self, c):
        """Cara utility index"""
        return -np.exp(-self.λ * c)
    
    def f(self, x):
        """Density of loss distribution"""
        return norm.pdf(x, loc=self.μ, scale=self.σ)

    def EU(self, p=0, x=0, n=12):
        """Expected utility of insurance contract with premium p 
        that pays a proportion x of the loss, 
        default is no insurance. Returns analytic solution but also
        numerical solution using Gauss-Hermite quadrature
        - n is the number of nodes in Gauss-Hermite quadrature"""
        
        λ , μ , σ , w0 = self.λ , self.μ , self.σ , self.w0

        # Analytic solution
        int_a = - np.exp(-λ*(w0-p))*np.exp(.5*(λ*(1-x)*σ)**2 + λ*(1-x)*μ)

        # Obtain vectors with nodes and weights
        x_w , w = roots_hermite(n)

        # Obtain vector of function evaluated at each x
        f = np.array([np.exp(λ*(1-x)*(np.sqrt(2)*σ*t + μ)) for t in x_w])
        int_n = - np.exp(-λ*(w0-p))*(1/np.sqrt(np.pi))*w@f
        return int_a , int_n
        
    def wtp(self, x=0, n=12):
        """Willingness to pay for insurance contract
        that pays a proportion x of the loss, default is no insurance.
        Returns analytic solution but also numerical solution using fsolve
        """
        
        λ , μ , σ, w0  = self.λ , self.μ , self.σ, self.w0
        
        # Analytic solution
        wtp_a = x*μ + .5*λ*σ**2*(1-(1-x)**2)

        # Define function for numerical solution
        def f(v):
            f = self.EU(p=v, x=x, n=n)[0] - self.EU(p=0, x=0, n=n)[0]
            return f

        # Set initial conditions and solve
        v0 = .1*w0 # Initial condition is 10% of wealth

        try:
            sol = fsolve(f, v0)
            return wtp_a , sol[0]
        
        except RuntimeError:
            return wtp_a, np.nan

**Example:** Create one consumer and calculate WTP and EU of contracts

In [20]:
c1 = Consumer()
EU = c1.EU()
print(f'Analytic calculation of EU when p=0 and x=0: {c1.EU()[0]}')
print(f'Numerical calcultion of EU when p=0 and x=0: {c1.EU()[1]}')
print(f'Analytic calculation of WTP when x=.2: {c1.wtp(x=.2)[0]}')
print(f'Numerical calcultion of WTP when x=.2: {c1.wtp(x=.2)[1]}')

Analytic calculation of EU when p=0 and x=0: -9.955206204452296e-07
Numerical calcultion of EU when p=0 and x=0: -9.955206204452298e-07
Analytic calculation of WTP when x=.2: 0.6324000000000001
Numerical calcultion of WTP when x=.2: 0.6323999999999997
